# Create and/or add data to tvde_earnings_history table in the bronze layer

In [1]:
# Importing libraries
import os
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import re
from datetime import datetime
from pyspark.sql import SparkSession
#from pyspark.sql.functions import col, lit, regexp_extract, when, to_date, unix_timestamp, from_unixtime, concat_ws, lpad, create_map
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, TimestampType



In [2]:
# layer and table
layer = "bronze"
table = "tvde_earnings_history"
catalog_name = "toll_reconciliation_tool"

# Define the desired warehouse location explicitly
project_dir = "C:/Users/renat/Documents/imgPdados-finance-uber/toll-reconciliation-tool/spark-warehouse"
warehouse_location = "file:///" + os.path.abspath(project_dir)
print(f"Setting Spark warehouse to: {warehouse_location}")

# Change the current working directory
os.chdir(project_dir)
print(f"Changed current working directory to: {os.getcwd()}")

Setting Spark warehouse to: file:///C:\Users\renat\Documents\imgPdados-finance-uber\toll-reconciliation-tool\spark-warehouse
Changed current working directory to: C:\Users\renat\Documents\imgPdados-finance-uber\toll-reconciliation-tool\spark-warehouse


In [3]:
# --- Functions (select_files, add_files_to_list, normalize_dataframe) ---
def select_files():
    """Opens a file selection dialog and returns a list with the paths of the selected files."""
    root = tk.Tk()  # Create a Tkinter window
    root.withdraw()  # Hide the main window

    file_paths = filedialog.askopenfilenames(
        title="Select files",  # Window title
    )

    return list(file_paths)  # Convert the returned tuple to a list

def add_files_to_list(file_list):
    """Adds the paths of the selected files to the list."""
    selected_file_paths = select_files()

    if selected_file_paths:  # Check if the user selected any files
        file_list.extend(selected_file_paths)  # Add the paths to the list
        print("Files added:")
        for file_path in selected_file_paths:
            print(file_path)
    else:
        print("No files selected.")

def normalize_dataframe(df, filename):
    """Normalizes a DataFrame according to specified rules."""

    df['Date'] = None
    df['Earnings'] = None
    df['Toll'] = None
    df['ServiceFee'] = None
    df['Tip'] = None
    df['StartTime'] = None
    df['TotalTime'] = None
    df['Distance'] = None
    df['DataInput'] = None

    toll_words = ["pedágio"]
    service_fee_words = ["taxa de serviço"]
    tip_words = ["valor extra"]

    # Extract year and month from filename
    year_match = re.search(r'\d{4}', filename)
    month_match = re.search(r'(jan|fev|mar|abr|mai|jun|jul|ago|set|out|nov|dez)', filename, re.IGNORECASE)

    if year_match and month_match:
        year = year_match.group(0)
        month = month_match.group(0).lower()
        month_number = {
            'jan': '01', 'fev': '02', 'mar': '03', 'abr': '04', 'mai': '05', 'jun': '06',
            'jul': '07', 'ago': '08', 'set': '09', 'out': '10', 'nov': '11', 'dez': '12'
        }[month]
    else:
        year = None
        month_number = None

    for index, row in df.iterrows():
        text = row['Extracted Text']

        # include control columns
        df.at[index, 'DataInput'] = datetime.now()

        # Date
        date_match = re.search(r'(seg|ter|qua|qui|sex|sáb|dom|Mon|Tue|Wed|Thu|Fri|Sat|Sun)?,? \d{2} de [a-z]{3}', text, re.IGNORECASE)
        if date_match:
            day_month = re.search(r'\d{2} de [a-z]{3}', date_match.group(0), re.IGNORECASE).group(0)
            day = re.search(r'\d{2}', day_month).group(0)
            if year and month_number:
                df.at[index, 'Date'] = f"{year}/{month_number}/{day}"
            else:
                df.at[index, 'Date'] = f"{day}" 
        else:
            df.at[index, 'Date'] = f'{year}/{month_number}/{day}'


        # Earnings
        earnings_match = re.search(r'€\s*([\d,.]+)', text)
        if earnings_match:
            df.at[index, 'Earnings'] = earnings_match.group(1).replace(',', '.')

        # Toll
        for word in toll_words:
            toll_match = re.search(r'([\d,.]+)\s*' + word, text, re.IGNORECASE)
            if toll_match:
                df.at[index, 'Toll'] = toll_match.group(1).replace(',', '.')
                break

        # Service Fee
        for word in service_fee_words:
            service_match = re.search(r'€\s*([\d,.]+)\s*' + word, text, re.IGNORECASE)
            if service_match:
                df.at[index, 'ServiceFee'] = service_match.group(1).replace(',', '.')
                break

        # Tip
        for word in tip_words:
            tip_match = re.search(r'([\d,.]+)\s*' + word, text, re.IGNORECASE)
            if tip_match:
                df.at[index, 'Tip'] = tip_match.group(1).replace(',', '.')
                break

        # Start Time
        start_time_match = re.search(r'(\d{2}-\d{2}|\d{2}\.\d{2}|\d{2}\*\d{2})', text)
        if start_time_match:
            df.at[index, 'StartTime'] = start_time_match.group(1).replace('.', ':').replace('*', ':').replace('-', ':')

        # Total Time
        total_time_match = re.search(r'(\d+)\s*min\s*(\d+)\s*seg', text)
        if total_time_match:
            df.at[index, 'TotalTime'] = f"{total_time_match.group(1)}:{total_time_match.group(2)}"

        # Distance
        distance_match = re.search(r'([\d,.]+)\s*km', text)
        if distance_match:
            df.at[index, 'Distance'] = distance_match.group(1).replace(',', '.')

    return df


In [4]:
# Selecting Files:
filepath_list = [] # Create an empty list
add_files_to_list(filepath_list) # Call the function to add files and print the final list
pandas_dataframes_list = []  # List to store DataFrames


Files added:
C:/Users/renat/Documents/imgPdados-finance-uber/toll-reconciliation-tool/2023-dezembro.csv


In [5]:
# puts each df in the 'dataframes_list', one for each CSV file.
for filepath in filepath_list:
    try:
        df = pd.read_csv(filepath, encoding='utf-8')  # Use utf-8 encoding
        pandas_dataframes_list.append(df)
        print(f"Successfully read {filepath}")
    except FileNotFoundError:
        print(f"Error: File not found - {filepath}")
    except pd.errors.EmptyDataError:
        print(f"Error: Empty CSV file - {filepath}")
    except pd.errors.ParserError:
        print(f"Error: Parsing error in {filepath}. Check the file format.")
    except Exception as e:
        print(f"An unexpected error occurred while reading {filepath}: {e}")



# You can access each DataFrame like this:
if pandas_dataframes_list:  # Check if the list is not empty
    for i,df in enumerate(pandas_dataframes_list):
        print(f"Number of rows in {df.shape}")
        print(f"\nFirst few rows of the {i+1}th DataFrame:")
        display(df.head())  # Print the first few rows of the first DataFrame
else:
    print("\nNo CSV files were successfully read.")


Successfully read C:/Users/renat/Documents/imgPdados-finance-uber/toll-reconciliation-tool/2023-dezembro.csv
Number of rows in (252, 1)

First few rows of the 1th DataFrame:


,Extracted Text
0,"sex , 29 de dez"
1,"€ 2,60 1l-46 Taxa de serviço"
2,"€3,23 21-23 UberX 6 min 22 segundos 138 km 4 €..."
3,"€5,22 UberX Saver . 18 min 46 segundos 20-12 8..."
4,"€4,37 UberX Saver ' 18 min 1l segundos 19.45 6..."


In [6]:
# normalizing Pandas DataFrame 
normalized_pandas_dataframes_list = [normalize_dataframe(df.copy(), os.path.basename(filepath)) for df, filepath in zip(pandas_dataframes_list, filepath_list)]


# Now 'normalized_dataframes' contains the normalized DataFrames.
# Configure pandas display options
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)  # Increase terminal width (adjust as needed)

# To view the first normalized DataFrame rows:
if normalized_pandas_dataframes_list:  # Check if the list is not empty
    for i,ndf in enumerate(normalized_pandas_dataframes_list):
        print(f"Number of rows in {ndf.shape}")
        print(f"\nFirst few rows of the {i+1}th DataFrame:")
        display(ndf.head())  # Print the first few rows of each DataFrame
else:
    print("\nNo files were successfully read.")


Number of rows in (252, 10)

First few rows of the 1th DataFrame:


,Extracted Text,Date,Earnings,Toll,ServiceFee,Tip,StartTime,TotalTime,Distance,DataInput
0,"sex , 29 de dez",2023/12/29,None,None,None,None,None,None,None,2025-05-03 22:06:41.212390
1,"€ 2,60 1l-46 Taxa de serviço",2023/12/29,2.60,None,None,None,None,None,None,2025-05-03 22:06:41.215383
2,"€3,23 21-23 UberX 6 min 22 segundos 138 km 4 €...",2023/12/29,3.23,None,None,0.50,21:23,6:22,138,2025-05-03 22:06:41.216381
3,"€5,22 UberX Saver . 18 min 46 segundos 20-12 8...",2023/12/29,5.22,None,None,None,20:12,18:46,8.22,2025-05-03 22:06:41.216381
4,"€4,37 UberX Saver ' 18 min 1l segundos 19.45 6...",2023/12/29,4.37,None,None,None,19:45,None,6.89,2025-05-03 22:06:41.217378


In [7]:
# Convertendo os tipos de dados no Pandas explicitamente
for ndf in normalized_pandas_dataframes_list:
    numeric_cols = ['Earnings', 'Toll', 'ServiceFee', 'Tip', 'Distance']
    for col in numeric_cols:
        ndf[col] = pd.to_numeric(ndf[col], errors='coerce')
    ndf['DataInput'] = pd.to_datetime(ndf['DataInput'])
    string_cols = ['Extracted Text', 'Date', 'StartTime', 'TotalTime']
    for col in string_cols:
        ndf[col] = ndf[col].astype(str)
    print(ndf.dtypes) # Verifique os tipos após a conversão

Extracted Text            object
Date                      object
Earnings                 float64
Toll                     float64
ServiceFee               float64
Tip                      float64
StartTime                 object
TotalTime                 object
Distance                 float64
DataInput         datetime64[ns]
dtype: object


In [8]:
# Initialize Spark session with the specified warehouse directory
spark = SparkSession.builder \
    .appName(catalog_name) \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.warehouse.dir", warehouse_location) \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .enableHiveSupport() \
    .getOrCreate()


# Confirming that the session is working
if spark:
    print("Spark session started successfully!")
    print(f"Application name: {spark.sparkContext.appName}")
else:
    print("Failed to start Spark session.")


Spark session started successfully!
Application name: toll_reconciliation_tool


In [9]:
try:
    # Show a list of layers/schemas/databases in the spark-warehouse
    spark.sql("SHOW DATABASES").show()
except Exception as e:
    print(f"Error: {e}")

# Check if the 'bronze' schema exists
result = spark.sql("SHOW SCHEMAS").collect()
schemas = [row[0] for row in result]

if "bronze" not in schemas:
    print("Error: The 'bronze' schema does not exist. Please create it.")  # error for not finding the bronze layer
else:
    print("Selecting the 'bronze' schema.")
    # Switch to the bronze layer
    spark.sql(f"USE {layer}")



+---------+
|namespace|
+---------+
|   bronze|
|  default|
|     gold|
|   silver|
+---------+

Selecting the 'bronze' schema.


In [10]:

# Convert each Pandas DataFrame in the list to a Spark DataFrame
spark_dataframes_list = [spark.createDataFrame(ndf) for ndf in normalized_pandas_dataframes_list]


In [11]:

# To view the first normalized Spark DataFrame rows:
if spark_dataframes_list:  # Check if the list is not empty
    for i, sdf in enumerate(spark_dataframes_list):
        print(f"Visualizando DataFrame número: {i + 1} da lista")
        print("Schema:")
        sdf.printSchema()
        #print("\nPrimeiras 5 linhas:")
        #sdf.head(5)
        print("-" * 30)  # Separador para melhor visualizaçã
else:
    print("\nNo files were successfully read.")

Visualizando DataFrame número: 1 da lista
Schema:
root
 |-- Extracted Text: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Earnings: double (nullable = true)
 |-- Toll: double (nullable = true)
 |-- ServiceFee: double (nullable = true)
 |-- Tip: double (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- TotalTime: string (nullable = true)
 |-- Distance: double (nullable = true)
 |-- DataInput: timestamp (nullable = true)

------------------------------


In [12]:
import sys
import subprocess
from pyspark.sql import SparkSession
import importlib.util
import os

def obter_versoes_notebook():
    """
    Obtém e imprime as versões do Python, Spark, Scala, Java e Delta Lake, projetado para execução em um ambiente de notebook Jupyter.
    """
    print("Obtendo informações de versão:")

    # 1. Versão do Python
    versao_python = sys.version
    print(f"Versão do Python: {versao_python}")

    # 2. Versão do Spark
    try:
        spark = SparkSession.builder.appName("VersaoSpark").getOrCreate()
        versao_spark = spark.version
        print(f"Versão do Spark: {versao_spark}")
    except Exception as e:
        print(f"Erro ao obter a versão do Spark: {e}")
        print(f"Detalhes do erro: {e}")
        versao_spark = None
    finally:
        if 'spark' in locals():
            spark.stop()

    # 3. Versão do Scala
    try:
        versao_scala = subprocess.run(['scala', '-version'], capture_output=True, text=True, check=True).stderr
        versao_scala = versao_scala.split("version ")[1].split(" ")[0]  # Extract Scala version
        print(f"Versão do Scala: {versao_scala}")
    except subprocess.CalledProcessError as e:
        print(f"Erro ao obter a versão do Scala: {e}")
        print(f"Detalhes do erro: {e}")
        versao_scala = None
    except FileNotFoundError:
        print("Scala não encontrado. Certifique-se de que o Scala está no seu PATH e que a variável de ambiente SCALA_HOME está definida corretamente.")
        if 'SCALA_HOME' in os.environ:
            print(f"Verifique se a variável de ambiente SCALA_HOME está definida corretamente: {os.environ['SCALA_HOME']}")
        else:
            print("A variável de ambiente SCALA_HOME não está definida. Defina-a para o diretório de instalação do Scala, por exemplo: C:\\Program Files\\Scala")
        versao_scala = "Scala não instalado"  # Atualizado para refletir a não instalação

    # 4. Versão do Java
    try:
        versao_java = subprocess.run(['java', '-version'], capture_output=True, text=True, check=True).stderr
        versao_java = versao_java.split("version ")[1].split("\n")[0].strip()  # Extract Java version
        print(f"Versão do Java: {versao_java}")
    except subprocess.CalledProcessError as e:
        print(f"Erro ao obter a versão do Java: {e}")
        print(f"Detalhes do erro: {e}")
        versao_java = "Erro ao executar 'java -version'" # Atualizado para refletir o erro na execução
    except FileNotFoundError:
        print("Java não encontrado. Certifique-se de que o Java está no seu PATH e que a variável de ambiente JAVA_HOME está definida corretamente.")
        if 'JAVA_HOME' in os.environ:
            print(f"Verifique se a variável de ambiente JAVA_HOME está definida corretamente: {os.environ['JAVA_HOME']}")
        else:
            print("A variável de ambiente JAVA_HOME não está definida. Defina-a para o diretório de instalação do Java, por exemplo: C:\\Program Files\\Java\\jdk-11.0.10")
        versao_java = "Java não instalado" # Atualizado para refletir a não instalação

    # 5. Versão do Delta Lake
    try:
        delta_spec = importlib.util.find_spec("delta")
        if delta_spec is not None:
            import delta
            versao_delta = delta.__version__
            print(f"Versão do Delta Lake: {versao_delta}")
        else:
            versao_delta = None
            print("Delta Lake não está instalado neste ambiente.")
            print("Instale-o usando 'pip install delta-spark' ou adicione o JAR do Delta Lake ao Spark.")
    except ImportError as e:
        versao_delta = None
        print("Delta Lake não está instalado neste ambiente. Certifique-se de que o pacote 'delta' está instalado corretamente.")
        print("Instale-o usando 'pip install delta-spark' ou verifique sua configuração do Spark, incluindo o seguinte:")
        print(f"  - Verifique se o pacote 'delta-spark' está instalado no ambiente Python correto (o mesmo que seu kernel do Jupyter Notebook).")
        print(f"  - Se você estiver usando o Spark com JARs, verifique se o JAR do Delta Lake está incluído na configuração do Spark (por exemplo, em 'spark.jars' ou '--jars').")
        print(f"Detalhes do erro: {e}")
    except AttributeError:
        versao_delta = None
        print("Erro ao obter a versão do Delta Lake. Verifique se a instalação está correta.")
        print("Certifique-se de que você instalou o Delta Lake usando 'pip install delta-spark' ou método equivalente, e que sua versão do Spark é compatível com a versão do Delta Lake.")
        print("Se você estiver usando o Spark com JARs, verifique se a versão do JAR do Delta Lake corresponde à versão do pacote 'delta-spark' instalado.")

    return {
        "versao_python": versao_python,
        "versao_spark": versao_spark,
        "versao_scala": versao_scala,
        "versao_java": versao_java,
        "versao_delta": versao_delta,
    }

# Chamando a função e armazenando os resultados
versoes = obter_versoes_notebook()

# Se você quiser usar as versões em outras células, você pode usar o dicionário 'versoes'
# Por exemplo:
# if versoes['versao_spark']:
#   print(f"Versão do Spark para uso posterior: {versoes['versao_spark']}")



Obtendo informações de versão:
Versão do Python: 3.11.4 (tags/v3.11.4:d2340ef, Jun  7 2023, 05:45:37) [MSC v.1934 64 bit (AMD64)]
Versão do Spark: 3.5.5
Scala não encontrado. Certifique-se de que o Scala está no seu PATH e que a variável de ambiente SCALA_HOME está definida corretamente.
Verifique se a variável de ambiente SCALA_HOME está definida corretamente: C:\Users\renat\AppData\Local\Coursier\data
Versão do Java: "1.6.0_38"
Erro ao obter a versão do Delta Lake. Verifique se a instalação está correta.
Certifique-se de que você instalou o Delta Lake usando 'pip install delta-spark' ou método equivalente, e que sua versão do Spark é compatível com a versão do Delta Lake.
Se você estiver usando o Spark com JARs, verifique se a versão do JAR do Delta Lake corresponde à versão do pacote 'delta-spark' instalado.


In [13]:
import os

path_env = os.environ['PATH']
print(f"PATH do Python: {path_env}")

if "scala" in path_env.lower():
    print("O PATH do Scala parece estar configurado.")
else:
    print("O PATH do Scala não está configurado corretamente para o Python.")

PATH do Python: c:\Users\renat\AppData\Local\Programs\Python\Python311;c:\Users\renat\AppData\Roaming\Python\Python311\Scripts;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.6\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.6\libnvvp;C:\Program Files\Java\jdk-17.0.2\bin;C:\Program Files\Common Files\Oracle\Java\javapath;C:\Program Files (x86)\Common Files\Oracle\Java\java8path;C:\Program Files (x86)\Common Files\Oracle\Java\javapath;C:\Windows\system32;C:\Windows;C:\Windows\System32\Wbem;C:\Windows\System32\WindowsPowerShell\v1.0\;C:\Windows\System32\OpenSSH\;C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common;C:\Program Files\PuTTY\;C:\ProgramData\chocolatey\bin;C:\Users\renat\AppData\Roaming\Python\Python311;C:\Users\renat\AppData\Roaming\Python\Python311\Scripts;C:\Users\renat\AppData\Local\Android\Sdk\platform-tools;C:\msys64\mingw64\bin;C:\Program Files\Git\cmd;C:\Program Files\NVIDIA Corporation\NVIDIA app\NvDLISR;C:\Program Files\Portugal Identity Card\;C:\

In [14]:
import os
import subprocess

os.environ['SCALA_HOME'] = r'C:\Users\renat\AppData\Local\Coursier\data' # Substitua pelo caminho correto
os.environ['PATH'] = os.environ['PATH'] + os.pathsep + os.path.join(os.environ['SCALA_HOME'], 'bin')

try:
    versao_scala = subprocess.run(['scala', '-version'], capture_output=True, text=True, check=True).stderr
    versao_scala = versao_scala.split("version ")[1].split(" ")[0]
    print(f"Versão do Scala: {versao_scala}")
except subprocess.CalledProcessError as e:
    print(f"Erro ao obter a versão do Scala: {e}")
    print(f"Detalhes do erro: {e}")
    versao_scala = None
except FileNotFoundError:
    print("Scala não encontrado.")
    versao_scala = "Scala não instalado" # Output atualizado para refletir o resultado obtido

print(f"Versão do Scala: {versao_scala}")



Scala não encontrado.
Versão do Scala: Scala não instalado


In [15]:
import os
import subprocess

scala_bat_path = r"C:\Users\renat\AppData\Local\Coursier\data\bin\scala.bat" # Substitua pelo caminho correto

try:
    resultado = subprocess.run([scala_bat_path, '-version'], capture_output=True, text=True, check=True)
    print(f"Saída: {resultado.stdout}")
except subprocess.CalledProcessError as e:
    print(f"Erro ao executar scala.bat: {e}")
    print(f"Saída do erro: {e.stderr}")
except FileNotFoundError:
    print(f"Arquivo scala.bat não encontrado em: {scala_bat_path}")

# Verificar o PATH que o Python está usando
print("Verificando o PATH do Python:")
print(os.environ['PATH'])

# Verificar se o diretório do Scala está no PATH
scala_dir = os.path.dirname(scala_bat_path)
if scala_dir in os.environ['PATH']:
    print(f"O diretório do Scala ({scala_dir}) está no PATH do Python.")
else:
    print(f"O diretório do Scala ({scala_dir}) não está no PATH do Python.")


Saída: Scala code runner version: 1.5.4
Scala version (default): 3.6.4

Verificando o PATH do Python:
c:\Users\renat\AppData\Local\Programs\Python\Python311;c:\Users\renat\AppData\Roaming\Python\Python311\Scripts;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.6\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.6\libnvvp;C:\Program Files\Java\jdk-17.0.2\bin;C:\Program Files\Common Files\Oracle\Java\javapath;C:\Program Files (x86)\Common Files\Oracle\Java\java8path;C:\Program Files (x86)\Common Files\Oracle\Java\javapath;C:\Windows\system32;C:\Windows;C:\Windows\System32\Wbem;C:\Windows\System32\WindowsPowerShell\v1.0\;C:\Windows\System32\OpenSSH\;C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common;C:\Program Files\PuTTY\;C:\ProgramData\chocolatey\bin;C:\Users\renat\AppData\Roaming\Python\Python311;C:\Users\renat\AppData\Roaming\Python\Python311\Scripts;C:\Users\renat\AppData\Local\Android\Sdk\platform-tools;C:\msys64\mingw64\bin;C:\Program Files\Git\cmd;C:\Program Fi

In [16]:
import os
import subprocess

scala_bat_path = r"C:\Users\renat\AppData\Local\Coursier\data\bin\scala.bat"  # Substitua pelo caminho correto

try:
    resultado = subprocess.run([scala_bat_path, '-version'], capture_output=True, text=True, check=True)
    versao_scala = resultado.stdout.strip()
    print(f"Saída completa do scala.bat -version: {versao_scala}")

    if "version" in versao_scala.lower():
        versao_scala = versao_scala.lower().split("version")[1].split()[0]
    elif ":" in versao_scala:
        versao_scala = versao_scala.split(":")[1].split()[0]
    elif "scala version" in versao_scala.lower():
        versao_scala = versao_scala.lower().split("scala version")[1].split()[0]
    else:
        print("Formato de versão desconhecido. Usando a saída bruta.")
        versao_scala = versao_scala  # Se não encontrar, usa a saída completa.

    print(f"Versão do Scala: {versao_scala}")

except subprocess.CalledProcessError as e:
    print(f"Erro ao executar scala.bat: {e}")
    print(f"Saída do erro: {e.stderr}")
    versao_scala = None
except FileNotFoundError:
    print(f"Arquivo scala.bat não encontrado em: {scala_bat_path}")
    versao_scala = "Scala não instalado"

print(f"Versão do Scala: {versao_scala}")

Saída completa do scala.bat -version: Scala code runner version: 1.5.4
Scala version (default): 3.6.4
Versão do Scala: :
Versão do Scala: :


In [17]:
for sdf in spark_dataframes_list:
    try:
        # Write the normalized Spark DataFrame to a Delta table
        sdf.write \
            .format('delta') \
            .mode('append') \
            .option('mergeSchema', 'true') \
            .saveAsTable(f'{layer}.{table}')
        print(f"Successfully processed and loaded {filepath} into {layer}.{table}")
            
    except Exception as e:
        print(f"An error occurred while processing {filepath}: {e}")
        import traceback
        traceback.print_exc()

print(f"All {len(filepath_list)} files processed (attempted) and loaded into '{layer}.{table}'.")


An error occurred while processing C:/Users/renat/Documents/imgPdados-finance-uber/toll-reconciliation-tool/2023-dezembro.csv: An error occurred while calling o78.saveAsTable.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.spark.sql.catalyst.catalog.SessionCatalog.validateTableLocation(SessionCatalog.scala:419)
	at org.apache.spar

Traceback (most recent call last):
  File "C:\Users\renat\AppData\Local\Temp\ipykernel_19180\898348527.py", line 8, in <module>
    .saveAsTable(f'{layer}.{table}')
     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\renat\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark\sql\readwriter.py", line 1586, in saveAsTable
    self._jwrite.saveAsTable(name)
  File "c:\Users\renat\AppData\Local\Programs\Python\Python311\Lib\site-packages\py4j\java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
                   ^^^^^^^^^^^^^^^^^
  File "c:\Users\renat\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark\errors\exceptions\captured.py", line 179, in deco
    return f(*a, **kw)
           ^^^^^^^^^^^
  File "c:\Users\renat\AppData\Local\Programs\Python\Python311\Lib\site-packages\py4j\protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: An error occurred while calling o78.saveAsTable.
: java.la

In [0]:
%sql
DESC DETAIL bronze.tvde_earnings_history

In [0]:
%sql
select *
from bronze.tvde_earnings_history

In [ ]:
# Stop the Spark session
spark.stop()